In [13]:
# Core
import time
import numpy as np
import joblib
import pandas as pd

# Models
import umap
import hdbscan

# From your script
from pathlib import Path
from datetime import datetime, timezone
from typing import Any, Dict, List, Optional

# Project imports (adjust path if needed)
import sys
sys.path.append("c:/Projects/clasificador_mensajes")
from _data_layer import registry
from _data_layer.registry import register_embedding_analysis, register_model

In [14]:
# ————————————————————————————————————————————————
# Notebook-friendly wrappers (keeps CLI working too)
# ————————————————————————————————————————————————
from typing import Union, Sequence, Tuple

def _in_notebook() -> bool:
    try:
        from IPython import get_ipython  # type: ignore
        return get_ipython() is not None
    except Exception:
        return False

def get_default_hdbscan_params() -> Dict[str, Any]:
    # Mirrors your interactive_menu defaults
    return {
        "pipeline_order": "umap-first",
        "mode": "new",
        "update_strategy": "retrain",
        # UMAP
        "n_neighbors": 15,
        "n_components": 2,
        "umap_metric": "cosine",
        # HDBSCAN
        "min_cluster_size": 2,
        "min_samples": 2,
        "cluster_selection_epsilon": 0.0,
        "cluster_selection_method": "eom",
        "hdb_metric": "euclidean",
    }

def get_params(num_vectors: int,
               overrides: Optional[Dict[str, Any]] = None,
               interactive: bool = False) -> Dict[str, Any]:
    """
    In notebooks: call with interactive=False and pass overrides to avoid input().
    In CLI: interactive=True to show your prompt menu.
    """
    if interactive and not _in_notebook():
        return interactive_menu(num_vectors)
    # non-interactive: start from defaults, apply overrides
    p = get_default_hdbscan_params()
    if overrides:
        p.update({k: v for k, v in overrides.items() if v is not None})
    return p

def _make_ui_index(batches: List[Dict[str, Any]],
                   gfilter: Optional[str],
                   mfilter: Optional[str]) -> Tuple[List[int], List[Dict[str, Any]]]:
    """Return (shown_ui_indices, batches) after printing the table."""
    shown = list_batches(batches, gfilter, mfilter)
    return shown, batches

def _resolve_selection(selection: Union[str, Sequence[int]],
                       shown: List[int]) -> List[int]:
    if isinstance(selection, str):
        if selection.strip().lower() == "all":
            return list(range(len(shown)))
        # support space-separated string in notebooks too
        return _parse_space_indices(selection, len(shown))
    # assume a list/tuple of UI indices
    out = []
    for i in selection:
        if not (0 <= int(i) < len(shown)):
            raise ValueError(f"UI index out of range: {i}")
        if int(i) not in out:
            out.append(int(i))
    return out

def select_and_run(selection: Union[str, Sequence[int]] = "all",
                   gfilter: Optional[str] = None,
                   mfilter: Optional[str] = None,
                   params_overrides: Optional[Dict[str, Any]] = None,
                   return_results: bool = True,
                   timeit: bool = False):
    """
    Notebook entrypoint.
    Example:
        shown, _ = _make_ui_index(fetch_batches(), gfilter=None, mfilter=None)
        select_and_run(selection=[0,2], params_overrides={'n_neighbors':30, 'n_components':5})
    """
    # 1) fetch & show
    batches = fetch_batches()
    shown, batches = _make_ui_index(batches, gfilter, mfilter)
    if not shown:
        print("(no batches match the current filters)")
        return None

    sel_ui = _resolve_selection(selection, shown)

    # 2) load & concat
    try:
        df, chosen_recs = _load_and_concat_selected(batches, shown, sel_ui)
    except SystemExit as e:
        print(str(e))
        return None

    gran_mix = sorted(set((_ensure_params_complete(r).get("granularity") or "?") for r in chosen_recs))
    models_mix = sorted(set((_ensure_params_complete(r).get("model") or "?") for r in chosen_recs))
    nrows = len(df)
    print(f"\n— Mix summary —  rows={nrows}  |  granularity={gran_mix}  |  model={models_mix}")
    print("  sources:", [r.get("id") for r in chosen_recs])

    # 3) params (non-interactive by default in notebooks)
    params = get_params(nrows, overrides=params_overrides, interactive=False)

    # 4) matrix
    X = np.vstack(df["embedding"].to_numpy()).astype("float64", copy=False)

    # 5) pipeline
    mode = params["mode"]; update_strategy = params["update_strategy"]
    umap_path      = MODEL_DIR / "umap_model.joblib"
    umap_proj_path = MODEL_DIR / "umap_proj_model.joblib"
    hdb_path       = MODEL_DIR / "hdbscan_model.joblib"

    # optional timing
    import time
    def _timed(fn, *a, **k):
        t0 = time.perf_counter(); out = fn(*a, **k); dt = time.perf_counter() - t0
        return out, dt

    # 5.1) UMAP → kD
    k = max(2, int(params["n_components"]))
    if mode == "update" and umap_path.exists():
        umap_model: umap.UMAP = joblib.load(umap_path)
        if update_strategy == "incremental":
            Z_k = umap_model.transform(X).astype("float64", copy=False)
            t_umap_k = 0.0
        else:
            X_full = np.vstack([umap_model._raw_data, X])  # type: ignore[attr-defined]
            if timeit:
                (umap_model, t_umap_k) = _timed(fit_umap, X_full, params["n_neighbors"], k, params["umap_metric"])
            else:
                umap_model = fit_umap(X_full, params["n_neighbors"], k, params["umap_metric"])
                t_umap_k = 0.0
            Z_k = umap_model.embedding_[-len(X):].astype("float64", copy=False)
    else:
        if timeit:
            (umap_model, t_umap_k) = _timed(fit_umap, X, params["n_neighbors"], k, params["umap_metric"])
        else:
            umap_model = fit_umap(X, params["n_neighbors"], k, params["umap_metric"])
            t_umap_k = 0.0
        Z_k = umap_model.embedding_.astype("float64", copy=False)

    if mode == "new" or update_strategy == "retrain":
        joblib.dump(umap_model, umap_path)

    # 5.2) HDBSCAN on kD
    if mode == "update" and hdb_path.exists():
        hdb_model: hdbscan.HDBSCAN = joblib.load(hdb_path)
        if update_strategy == "incremental":
            labels, strengths = hdbscan.approximate_predict(hdb_model, Z_k)
            df["cluster"] = labels; df["strength"] = strengths
            t_hdb = 0.0
        else:
            if timeit:
                (hdb_model, t_hdb) = _timed(
                    fit_hdbscan, Z_k,
                    params["min_cluster_size"], params["min_samples"],
                    params["cluster_selection_epsilon"], params["cluster_selection_method"],
                    params["hdb_metric"]
                )
            else:
                hdb_model = fit_hdbscan(
                    Z_k,
                    params["min_cluster_size"], params["min_samples"],
                    params["cluster_selection_epsilon"], params["cluster_selection_method"],
                    params["hdb_metric"]
                )
                t_hdb = 0.0
            df["cluster"]  = hdb_model.labels_
            df["strength"] = hdb_model.probabilities_
    else:
        if timeit:
            (hdb_model, t_hdb) = _timed(
                fit_hdbscan, Z_k,
                params["min_cluster_size"], params["min_samples"],
                params["cluster_selection_epsilon"], params["cluster_selection_method"],
                params["hdb_metric"]
            )
        else:
            hdb_model = fit_hdbscan(
                Z_k,
                params["min_cluster_size"], params["min_samples"],
                params["cluster_selection_epsilon"], params["cluster_selection_method"],
                params["hdb_metric"]
            )
            t_hdb = 0.0
        df["cluster"]  = hdb_model.labels_
        df["strength"] = hdb_model.probabilities_

    if mode == "new" or update_strategy == "retrain":
        joblib.dump(hdb_model, hdb_path)

    # 5.3) UMAP → 2D projection (for plotting only)
    if mode == "update" and umap_proj_path.exists() and update_strategy == "incremental":
        umap_proj: umap.UMAP = joblib.load(umap_proj_path)
        embedding_2d = umap_proj.transform(Z_k).astype("float64", copy=False)
        t_umap_2d = 0.0
    else:
        if timeit:
            (umap_proj, t_umap_2d) = _timed(
                fit_umap, Z_k,
                n_neighbors=max(30, params["n_neighbors"] // 2),
                n_components=2, metric="euclidean"
            )
        else:
            umap_proj = fit_umap(Z_k, n_neighbors=max(30, params["n_neighbors"] // 2), n_components=2, metric="euclidean")
            t_umap_2d = 0.0
        embedding_2d = umap_proj.embedding_.astype("float64", copy=False)

    if mode == "new" or update_strategy == "retrain":
        joblib.dump(umap_proj, umap_proj_path)

    # 6) persist experiment & register
    df = df.copy()
    if "embedding" in df.columns:
        del df["embedding"]
    df["umap_x"], df["umap_y"] = embedding_2d[:, 0], embedding_2d[:, 1]

    ts = datetime.now(timezone.utc).strftime("%Y%m%dT%H%M%SZ")
    out_path = EXPER_DIR / f"exp_mix_{ts}.joblib"
    joblib.dump({"df": df,
                 "source_batch_ids": [r.get("id") for r in chosen_recs],
                 "params": params},
                out_path, compress=3)
    print(f"\n✅ Results saved → {out_path.name}")

    try:
        umap_model_id = register_model(
            data_ref=str(umap_path),
            parameters={
                "n_neighbors": params["n_neighbors"],
                "n_components": params["n_components"],
                "metric": params["umap_metric"],
            },
            parents=[r.get("id") for r in chosen_recs],
            model_type="umap"
        )
        hdbscan_model_id = register_model(
            data_ref=str(hdb_path),
            parameters={
                "min_cluster_size":          params["min_cluster_size"],
                "min_samples":               params["min_samples"],
                "cluster_selection_epsilon": params["cluster_selection_epsilon"],
                "cluster_selection_method":  params["cluster_selection_method"],
                "metric":                    params["hdb_metric"],
            },
            parents=[r.get("id") for r in chosen_recs],
            model_type="hdbscan"
        )
        register_embedding_analysis(
            data_ref=str(out_path),
            parameters={
                "pipeline_order": params["pipeline_order"],
                "mode": mode,
                "update_strategy": update_strategy,
                "umap_model": umap_model_id,
                "hdbscan_model": hdbscan_model_id,
                "granularity_mix": sorted(set((_ensure_params_complete(r).get("granularity") or "?") for r in chosen_recs)),
                "source_models": sorted(set((_ensure_params_complete(r).get("model") or "?") for r in chosen_recs)),
                "source_n_vectors_total": int(len(df)),
                "source_batches": [r.get("batches") for r in chosen_recs],
                "source_batch_ids": [r.get("id") for r in chosen_recs],
            },
            parents=[r.get("id") for r in chosen_recs]
        )
        print("🧭 Registered analysis with lineage to the selected vector batches.")
    except Exception as e:
        print("⚠️ Registry registration failed (continuing):", e)

    if timeit and (t_umap_k or t_hdb or t_umap_2d):
        total = (t_umap_k + t_hdb + t_umap_2d)
        print(f"\n⏱ Timing: UMAP_kD={t_umap_k:.2f}s  |  HDBSCAN={t_hdb:.2f}s  |  UMAP_2D={t_umap_2d:.2f}s  |  total={total:.2f}s")

    return (df, str(out_path), str(umap_path), str(hdb_path), str(umap_proj_path)) if return_results else None


In [16]:

# Keep your CLI for terminals
def main():
    gfilter: Optional[str] = None  # "message" | "conversation" | None
    mfilter: Optional[str] = None  # substring on model

    # If running inside a notebook, bail out early to avoid input() loops
    if _in_notebook():
        print("Notebook detected — use select_and_run(...) instead of main().")
        return

    while True:
        batches = fetch_batches()
        shown = list_batches(batches, gfilter, mfilter)

        print("\nOptions:")
        print("  [space-separated indices] Analyze those batches (e.g., '1 3 7' or 'all')")
        print("  g) Set granularity filter (message / conversation / all)")
        print("  m) Set model filter (substring, empty to clear)")
        print("  r) Refresh list")
        print("  q) Quit")

        choice = input("Select: ").strip().lower()
        if choice == "q":
            return
        elif choice == "r":
            continue
        elif choice == "g":
            val = input("Granularity (message / conversation / all): ").strip().lower()
            gfilter = val if val in ("message", "conversation") else None
            continue
        elif choice == "m":
            val = input("Model filter substring (blank to clear): ").strip()
            mfilter = val or None
            continue
        else:
            if choice == "all":
                sel_ui = list(range(len(shown)))
            else:
                try:
                    sel_ui = _parse_space_indices(choice, len(shown))
                except ValueError as e:
                    print("Selection error:", e)
                    continue

            try:
                df, chosen_recs = _load_and_concat_selected(batches, shown, sel_ui)
            except SystemExit as e:
                print(str(e)); input("\n(Press Enter to go back)"); continue

            gran_mix = sorted(set((_ensure_params_complete(r).get("granularity") or "?") for r in chosen_recs))
            models_mix = sorted(set((_ensure_params_complete(r).get("model") or "?") for r in chosen_recs))
            nrows = len(df)
            print(f"\n— Mix summary —  rows={nrows}  |  granularity={gran_mix}  |  model={models_mix}")
            print("  sources:", [r.get("id") for r in chosen_recs])

            params = interactive_menu(nrows)

            X = np.vstack(df["embedding"].to_numpy()).astype("float64", copy=False)

            # (the rest of your original run body… you already have it below)
            # Tip: you can reuse select_and_run here if you want, but I left your original CLI intact.

In [19]:
# --- Define helpers (copied from your script) ---
def fit_umap(X: np.ndarray, n_neighbors: int, n_components: int, metric: str) -> umap.UMAP:
    return umap.UMAP(
        n_neighbors=n_neighbors,
        n_components=n_components,
        metric=metric,
        random_state=42
    ).fit(X)

def fit_hdbscan(X: np.ndarray,
                min_cluster_size: int,
                min_samples: int,
                cluster_selection_epsilon: float,
                cluster_selection_method: str,
                metric: str) -> hdbscan.HDBSCAN:
    return hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=min_samples,
        cluster_selection_epsilon=cluster_selection_epsilon,
        cluster_selection_method=cluster_selection_method,
        metric=metric,
        algorithm="generic",
        prediction_data=True,
    ).fit(X)

# --- Timing helper ---
def timeit(func, *args, **kwargs):
    import time
    t0 = time.perf_counter()
    result = func(*args, **kwargs)
    t1 = time.perf_counter()
    print(f"{func.__name__} took {t1 - t0:.3f} seconds")
    return result


In [17]:
import time

def timeit(func, *args, **kwargs):
    t0 = time.perf_counter()
    result = func(*args, **kwargs)
    t1 = time.perf_counter()
    print(f"{func.__name__} took {t1 - t0:.3f} seconds")
    return result


In [25]:
# 1) Load a batch (adapt path to your data)
import joblib
obj = joblib.load("path/to/your/batch.joblib")
df = obj["df"] if isinstance(obj, dict) else obj

# 2) Build X matrix
import numpy as np
X = np.vstack(df["embedding"].to_numpy()).astype("float64", copy=False)

# 3) Time UMAP and HDBSCAN
umap_model = timeit(fit_umap, X, n_neighbors=15, n_components=2, metric="cosine")
Z = umap_model.embedding_

hdb_model = timeit(
    fit_hdbscan,
    Z,
    min_cluster_size=2,
    min_samples=2,
    cluster_selection_epsilon=0.0,
    cluster_selection_method="eom",
    metric="euclidean"
)

labels = hdb_model.labels_
strengths = hdb_model.probabilities_


FileNotFoundError: [Errno 2] No such file or directory: 'path/to/your/batch.joblib'